In [2]:
import glob
import time
import pandas as pd
import numpy as np
import os
import platform
from pathlib import Path
import matplotlib.pyplot as plt
#astropy imports
from astropy.io import fits
from astropy.visualization import simple_norm
from astropy.stats import sigma_clipped_stats
from astropy.nddata import NDData
import ast
from astropy.nddata import StdDevUncertainty
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, Column
from astropy.utils.data import get_pkg_data_filename
from astropy.wcs import WCS
from photutils.detection import find_peaks
from astropy.nddata.utils import Cutout2D
from photutils.background import MMMBackground
from photutils.psf import extract_stars
# photutils imports
from photutils.detection import DAOStarFinder
from photutils.psf import PSFPhotometry, SourceGrouper, IntegratedGaussianPRF, extract_stars, EPSFBuilder, EPSFStar
# my modules
from cat_match import create_skycoord
from psf_gen import return_filtered_table, build_psf_from_catalog, save_psf_to_fits

os_check = platform.platform(terse=True)[:5]

if os_check == 'macOS':
    preamble = '/path/to/mac/'
    root2 = f'{preamble}Deconvolution/'
    root1 = f'{root2}Data/PHOTOMETRY/PHOTOM_CATS/'
else:
    preamble = '/path/to/linux/'
    root2 = f'{preamble}Deconvolution/'
    root1 = f'{root2}Data/PHOTOMETRY/PHOTOM_CATS/'

psf_dir = f'{root2}PSFs/'    
test_dir = f'{root2}test_dir'

cluster_images = pd.read_csv('final_file_merge_w_psf.csv') 
cluster_images = pd.read_csv('psfm_final_merge.csv') 
#cluster_images.iloc[18:20]['Band']
clusters = cluster_images['Cluster'].unique()
remove_clust = [ 'SpARCS0034', 'SpARCS0036','SpARCS0215', 'SpARCS1047', 'SpARCS1613']
clusters = [x for x in clusters if x not in remove_clust]
plot_dir = f'{root2}Plots'
star_cut_cats_dir = f'{root2}star_cut_catalogs'
#cluster_df_2 = pd.read_csv('final_merge.csv')
cluster_df_2 = cluster_images
clusters

['SPT0205',
 'SPT0546',
 'SPT2106',
 'SpARCS0035',
 'SpARCS0219',
 'SpARCS0335',
 'SpARCS1034',
 'SpARCS1051',
 'SpARCS1616',
 'SpARCS1634',
 'SpARCS1638']

In [16]:
def make_cutouts(cluster, band, catalog_file, data_file, noise_file, in_cluster = False):
    """
	Build a PSF using stars from a catalog by extracting RA/Dec, converting to pixels,
	and using EPSFBuilder.
	
	Parameters:
	- cluster: The cluster name
	- band: The band of the image
	- fits_file: Path to the FITS file containing image data
	- catalog_file: Path to the catalog (CSV or similar) containing RA and Dec columns
	- ra_col: Name of the column containing RA values in the catalog
	- dec_col: Name of the column containing Dec values in the catalog
	- size: Size of the cutout box for stars (default is 33x33 pixels)
	- threshold: Detection threshold for find_peaks (default is 500.0)
	
	Returns:
	- epsf: The built effective PSF object
	"""
	# Load the image data from the FITS file
    size = (32,32)
    main_dir = '/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/'
    #main_dir = '/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/test_dir/'
    with fits.open(data_file) as hdu_d:
        image_data = hdu_d[0].data
        image_header = hdu_d[0].header
        wcs = WCS(image_header)
        
      # Load the noise data from the FITS file
    with fits.open(noise_file) as hdu_n:
        noise_data = hdu_n[0].data
        noise_header = hdu_n[0].header
    # Load the catalog data (assuming it's a CSV; adjust this if using a different format)
    catalog = pd.read_csv(catalog_file)
    
    cid_m,cluster_m, zspec_m,zq_spec,M,D4000,eD4000,BIC,member,zcluster,UVspec,VJspec, NUVMINV, specid_m,zphot_m,Star_m,K_flag_m = catalog[['cluster_id','Cluster' ,'Redshift','Redshift_Quality','Mstellar','D4000','eD4000','delta_BIC','member','Redshift_c','UMINV','VMINJ','NUVMINV','SPECID','zphot','Star','K_flag_x']].values.T
    select_s = np.where((Star_m == 1) & (cluster_m == f'{cluster}'))
    
    
    galaxy_catalog = catalog.iloc[select_g]
    star_catalog = catalog.iloc[select_s]
    coords = [(np.array(ra), np.array(dec)) for ra, dec in zip(galaxy_catalog['ra_x'], galaxy_catalog['dec_x'])]

    for i in range(len(galaxy_catalog)):
        gal_id = galaxy_catalog.iloc[i]['cPHOTID']
        position = SkyCoord(coords[i][0], coords[i][1], unit=(u.deg, u.deg), frame='icrs')
        
        # Data cutout
        cutout_data =  Cutout2D(image_data, position,  size, wcs=wcs)
        # Noise cutouts
        cutout_noise = Cutout2D(noise_data, position, size, wcs=wcs)
        
        #hdud = fits.PrimaryHDU(cutout_data, header=image_header)
        #hdun = fits.PrimaryHDU(cutout_noise, header=noise_header)

        hdud = fits.PrimaryHDU(cutout_data.data, header=image_header)
        hdun = fits.PrimaryHDU(cutout_noise.data, header=noise_header)
        
        data_cutout_filename = f'{cluster}_{band}_{gal_id}_data.fits'
        noise_cutout_filename = f'{cluster}_{band}_{gal_id}_noise.fits'
    
        data_cutout_path = f'{main_dir}{cluster}/{band}/data_cutouts/'
        noise_cutout_path =  f'{main_dir}{cluster}/{band}/noise_cutouts/'

        #os.makedirs(data_cutout_path, exist_ok=True)
        #os.makedirs(noise_cutout_path, exist_ok=True)
        
        data_cutout = os.path.join(data_cutout_path,data_cutout_filename)
        noise_cutout = os.path.join(noise_cutout_path,noise_cutout_filename)

        hdud.writeto(data_cutout, overwrite=True)
        hdun.writeto(noise_cutout, overwrite=True)
    
        fits.setval(data_cutout, 'CRVAL1', value = galaxy_catalog.iloc[i]['ra_x'])
        fits.setval(data_cutout, 'CRVAL2', value = galaxy_catalog.iloc[i]['dec_x'])
        fits.setval(data_cutout, 'CRPIX1', value = size[0])
        fits.setval(data_cutout, 'CRPIX2', value = size[0])
        
        fits.setval(noise_cutout, 'CRVAL1', value = galaxy_catalog.iloc[i]['ra_x'])
        fits.setval(noise_cutout, 'CRVAL2', value = galaxy_catalog.iloc[i]['dec_x'])
        fits.setval(noise_cutout, 'CRPIX1', value = size[0])
        fits.setval(noise_cutout, 'CRPIX2', value = size[0])

In [3]:
def make_cutouts_psfm(cluster, band, catalog_file, data_file):
    """
	Build a PSF using stars from a catalog by extracting RA/Dec, converting to pixels,
	and using EPSFBuilder.
	
	Parameters:
	- cluster: The cluster name
	- band: The band of the image
	- fits_file: Path to the FITS file containing image data
	- catalog_file: Path to the catalog (CSV or similar) containing RA and Dec columns
	- ra_col: Name of the column containing RA values in the catalog
	- dec_col: Name of the column containing Dec values in the catalog
	- size: Size of the cutout box for stars (default is 33x33 pixels)
	- threshold: Detection threshold for find_peaks (default is 500.0)
	
	Returns:
	- epsf: The built effective PSF object
	"""
	# Load the image data from the FITS file
    size = (32,32)
    main_dir = f'{root2}Deconvolution/'
 
    with fits.open(data_file) as hdu_d:
        image_data = hdu_d[0].data
        image_header = hdu_d[0].header
        wcs = WCS(image_header)
        
   
    # Load the catalog data (assuming it's a CSV; adjust this if using a different format)
    catalog = pd.read_csv(catalog_file)
    
    cid_m,cluster_m, zspec_m,zq_spec,M,D4000,eD4000,BIC,member,zcluster,UVspec,VJspec, NUVMINV, specid_m,zphot_m,Star_m,K_flag_m = catalog[['cluster_id','Cluster' ,'Redshift','Redshift_Quality','Mstellar','D4000','eD4000','delta_BIC','member','Redshift_c','UMINV','VMINJ','NUVMINV','SPECID','zphot','Star','K_flag_x']].values.T
    select_s = np.where((Star_m == 1) & (cluster_m == f'{cluster}'))
    select_g = np.where((Star_m == 0) & (cluster_m == f'{cluster}'))
    
    galaxy_catalog = catalog.iloc[select_g]
    star_catalog = catalog.iloc[select_s]
    coords = [(np.array(ra), np.array(dec)) for ra, dec in zip(galaxy_catalog['ra_x'], galaxy_catalog['dec_x'])]

    for i in range(len(galaxy_catalog)):
        gal_id = galaxy_catalog.iloc[i]['cPHOTID']
        position = SkyCoord(coords[i][0], coords[i][1], unit=(u.deg, u.deg), frame='icrs')
        
        # Data cutout
        cutout_data =  Cutout2D(image_data, position,  size, wcs=wcs)
     

        hdud = fits.PrimaryHDU(cutout_data.data, header=image_header)
        
        
        data_cutout_filename = f'{cluster}_{band}_{gal_id}_psf_data_cutouts.fits'
        
    
        data_cutout_path = f'{main_dir}{cluster}/{band}/psf_matched_fits/'
        

        #os.makedirs(data_cutout_path, exist_ok=True)
        #os.makedirs(noise_cutout_path, exist_ok=True)
        
        data_cutout = os.path.join(data_cutout_path,data_cutout_filename)
        

        hdud.writeto(data_cutout, overwrite=True)
        
    
        fits.setval(data_cutout, 'CRVAL1', value = galaxy_catalog.iloc[i]['ra_x'])
        fits.setval(data_cutout, 'CRVAL2', value = galaxy_catalog.iloc[i]['dec_x'])
        fits.setval(data_cutout, 'CRPIX1', value = size[0])
        fits.setval(data_cutout, 'CRPIX2', value = size[0])
        

In [17]:
t0 = time.time()

In [18]:
for cluster in clusters:

    file = f'{root2}merged_catalogs/{cluster}_gg_sextractor_merged.csv'
    cluster_df = pd.read_csv(file)
    #cluster_df_2 = pd.read_csv(files_bands_list)
    #filtered_df = cluster_df_2[cluster_df_2['Cluster'] == cluster]
    
    filtered_df = cluster_df_2[(cluster_df_2['Cluster'] == cluster) & (cluster_df_2['Band'] == 'FOURSTARJ')]

    #band_list = list(filtered_df['Band'])
    
    for i in filtered_df.index:
        #data = filtered_df['PSFM Data File'][i]
        data = filtered_df['Data File'][i]
        noise = filtered_df['Noise File'][i]
        band = filtered_df['Band'][i]
        #print(filtered_df['Cluster'][i], cluster_df,filtered_df['Band'][i], filtered_df['Data File'][i], filtered_df['Noise File'][i])
        make_cutouts(cluster, band, file, data, noise)

In [ ]:
t1 = time.time()
total_time = (t1 - t0)/60
print(f'Total time: {total_time} minutes')

In [ ]:
for cluster in clusters:

    file = f'{root2}merged_catalogs/{cluster}_gg_sextractor_merged.csv'
    cluster_df = pd.read_csv(file)
    #cluster_df_2 = pd.read_csv(files_bands_list)
    #filtered_df = cluster_df_2[cluster_df_2['Cluster'] == cluster]
    
    filtered_df = cluster_df_2[(cluster_df_2['Cluster'] == cluster) & (cluster_df_2['Band'] == 'FOURSTARJ')]

    #band_list = list(filtered_df['Band'])
    
    for i in filtered_df.index:
        data = filtered_df['PSFM Data File'][i]
        data = filtered_df['PSFM Data File'][i]
        #noise = filtered_df['Noise File'][i]
        band = filtered_df['Band'][i]
        #print(filtered_df['Cluster'][i], cluster_df,filtered_df['Band'][i], filtered_df['Data File'][i], filtered_df['Noise File'][i])
        make_cutouts_psfm(cluster, band, file, data)

# HST Cutouts (testing)

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.nddata import Cutout2D
import numpy as np
import pandas as pd

def find_common_footprint(fits_files):
    """
    Find the common footprint of multiple FITS images by intersecting their WCS bounds.
    """
    footprints = []
    
    for file in fits_files:
        with fits.open(file) as hdul:
            wcs = WCS(hdul[0].header)
            ny, nx = hdul[0].data.shape
            corners = SkyCoord.from_pixel([0, 0, nx-1, nx-1], [0, ny-1, 0, ny-1], wcs)
            footprints.append(corners)
    
    # Intersect footprint regions (bounding RA and DEC ranges)
    ra_min = max([fp.ra.min().deg for fp in footprints])
    ra_max = min([fp.ra.max().deg for fp in footprints])
    dec_min = max([fp.dec.min().deg for fp in footprints])
    dec_max = min([fp.dec.max().deg for fp in footprints])
    
    return ra_min, ra_max, dec_min, dec_max

def filter_coordinates_in_footprint(ra_min, ra_max, dec_min, dec_max, coords_df):
    """
    Filter coordinates that lie within the common footprint.
    """
    filtered_coords = coords_df[(coords_df['ra'] >= ra_min) & (coords_df['ra'] <= ra_max) & 
                                (coords_df['dec'] >= dec_min) & (coords_df['dec'] <= dec_max)]
    return filtered_coords

def create_cutouts(fits_files, filtered_coords, cutout_size=(50, 50)):
    """
    Create cutouts for each coordinate in the filtered list within each FITS file.
    """
    cutouts_dict = {f"cutouts_band_{i+1}": [] for i in range(len(fits_files))}
    
    for i, file in enumerate(fits_files):
        with fits.open(file) as hdul:
            wcs = WCS(hdul[0].header)
            for _, row in filtered_coords.iterrows():
                position = SkyCoord(ra=row['ra'], dec=row['dec'], unit='deg')
                cutout = Cutout2D(hdul[0].data, position, cutout_size, wcs=wcs)
                cutouts_dict[f"cutouts_band_{i+1}"].append(cutout.data)
    
    return cutouts_dict

# Example usage:
# Assuming coords_df is a DataFrame with 'ra' and 'dec' columns
fits_files = ['file1.fits', 'file2.fits', 'file3.fits']
coords_df = pd.DataFrame({'ra': [150.0, 151.2, 149.8], 'dec': [2.3, 2.5, 2.1]})

# Step 1: Find common footprint
ra_min, ra_max, dec_min, dec_max = find_common_footprint(fits_files)

# Step 2: Filter coordinates within the common footprint
filtered_coords = filter_coordinates_in_footprint(ra_min, ra_max, dec_min, dec_max, coords_df)

# Step 3: Create cutouts
cutouts_dict = create_cutouts(fits_files, filtered_coords)

# Converting filtered_coords to an Astropy table to return
filtered_table = Table.from_pandas(filtered_coords)
print(filtered_table)
